# Overview

- nb004で作成した特徴量を使ってrfcモデルを作成する。

# Const

In [2]:
NB = '006'
N_FOLD = 5
FEATSETS = 'nb004_librosa_mfcc.csv'

DIR_AUDIO = './../data_ignore/official/train_audio'
DIR_MODEL = './../data_ignore/model'
PATH_FEAT = f'./../data_ignore/features/table/{FEATSETS}'
PATH_TRAIN_CSV = './../data_ignore/official/train.csv'
PATH_TEST_CSV = './../data/external_dataset/birdcall-check/test.csv'
DIR_TEST_AUDIO = './../data/external_dataset/birdcall-check/test_audio/'

# Import everything I need :)

In [27]:
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import sys
import glob
import time
import pickle
import joblib
import librosa
import numpy as np
import pandas as pd
from fastprogress import progress_bar
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

sys.path.insert(0, './../src/util/')
from const import BIRD_CODE, INV_BIRD_CODE

# My function

In [9]:
def extract_mfccs(x, sr, n_fft, hop_length, n_mels):
    mfccs = librosa.feature.mfcc(y=x,
                                 sr=sr,
                                 n_fft=n_fft,
                                 hop_length=hop_length,
                                 center=False,
                                 n_mels=n_mels)
    feat_name = [f'librosa_mfcc_{i+1}' for i in range(n_mels)]
    return mfccs.T, feat_name

# Preparation

setting

In [3]:
pd.set_option('display.max_columns', 100)

<br>

loading

In [7]:
train_csv = pd.read_csv(PATH_TRAIN_CSV)
test_csv = pd.read_csv(PATH_TEST_CSV)
df_feat = pd.read_csv(PATH_FEAT)
print(df_feat.shape)
df_feat.head()

(21375, 13)


,librosa_mfcc_2,librosa_mfcc_3,librosa_mfcc_4,librosa_mfcc_5,librosa_mfcc_6,librosa_mfcc_7,librosa_mfcc_8,librosa_mfcc_9,librosa_mfcc_10,librosa_mfcc_11,librosa_mfcc_12,birds,filename
0,6.57,-29.890,7.2200,-0.8910,1.132,-1.086,0.7397,1.1660,0.4950,-0.8760,0.2184,aldfly,XC134874.mp3
1,21.92,-13.020,5.3240,5.5660,-0.261,-3.242,1.2250,1.0030,1.7480,-1.1920,-0.1931,aldfly,XC135454.mp3
2,16.45,-14.470,8.0800,4.5700,-0.888,-3.943,2.5800,0.5890,0.9697,-0.3743,-0.7420,aldfly,XC135455.mp3
3,19.39,-12.805,6.5940,6.4180,-0.856,-4.598,1.6750,1.8410,1.8660,-1.0560,-0.8057,aldfly,XC135456.mp3
4,30.81,-3.867,0.5264,0.6313,-0.657,-3.547,1.3420,0.1382,1.0670,-0.0874,0.5970,aldfly,XC135457.mp3


## create X, y

In [14]:
y = df_feat['birds'].apply(lambda x: BIRD_CODE[x]).values
X = df_feat.drop(['birds', 'filename'], axis=1)

In [6]:
print(y)
X.head()

[  0   0   0 ... 263 263 263]


,librosa_mfcc_2,librosa_mfcc_3,librosa_mfcc_4,librosa_mfcc_5,librosa_mfcc_6,librosa_mfcc_7,librosa_mfcc_8,librosa_mfcc_9,librosa_mfcc_10,librosa_mfcc_11,librosa_mfcc_12
0,6.57,-29.890,7.2200,-0.8910,1.132,-1.086,0.7397,1.1660,0.4950,-0.8760,0.2184
1,21.92,-13.020,5.3240,5.5660,-0.261,-3.242,1.2250,1.0030,1.7480,-1.1920,-0.1931
2,16.45,-14.470,8.0800,4.5700,-0.888,-3.943,2.5800,0.5890,0.9697,-0.3743,-0.7420
3,19.39,-12.805,6.5940,6.4180,-0.856,-4.598,1.6750,1.8410,1.8660,-1.0560,-0.8057
4,30.81,-3.867,0.5264,0.6313,-0.657,-3.547,1.3420,0.1382,1.0670,-0.0874,0.5970


## Train

In [7]:
seed = 7
model_params = {'n_estimators': 100,
                'random_state': seed,
                'n_jobs': 5
               }

In [8]:
%%time
kf = KFold(n_splits=N_FOLD, shuffle=True, random_state=seed)
models = []
scores_train = []
scores_valid = []
oof_pred = np.zeros(len(X))
for fold_i, (train_idx, valid_idx) in enumerate(kf.split(X)):
    print(f'===== fold {fold_i+1}/{N_FOLD}  {time.ctime()} =====')
    X_train, X_valid = X.values[train_idx, :], X.values[valid_idx, :]
    y_train, y_valid = y[train_idx], y[valid_idx]
    
    # init
    model = RandomForestClassifier(**model_params)
    
    # fit
    model.fit(X_train, y_train)
    models.append(model)
    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)
    oof_pred[valid_idx] = y_valid_pred
    
    # evaluate
    acc_train = accuracy_score(y_train, y_train_pred)
    acc_valid = accuracy_score(y_valid, y_valid_pred)
    scores_train.append(acc_train)
    scores_valid.append(acc_valid)
    
    print(f':: score(train): {acc_train :.5f}, score(valid) {acc_valid :.5f} ::\n')
    
scores_train = np.array(scores_train)
scores_valid = np.array(scores_valid)
score_oof = accuracy_score(y, oof_pred)
print(f'===== finish {time.ctime} =====')
print(f':: score(train): mean={scores_train.mean():.5f}, std={scores_train.std():.5f} ::')
print(f':: score(valid): mean={scores_valid.mean():.5f}, std={scores_valid.std():.5f} ::')
print(f':: score(oof): {score_oof:.5f} ::')

===== fold 1/5  Sun Aug  2 19:41:57 2020 =====
:: score(train): 0.99971, score(valid) 0.12117 ::

===== fold 2/5  Sun Aug  2 19:42:06 2020 =====
:: score(train): 0.99982, score(valid) 0.12327 ::

===== fold 3/5  Sun Aug  2 19:42:14 2020 =====
:: score(train): 0.99977, score(valid) 0.12678 ::

===== fold 4/5  Sun Aug  2 19:42:23 2020 =====
:: score(train): 0.99977, score(valid) 0.12678 ::

===== fold 5/5  Sun Aug  2 19:42:31 2020 =====
:: score(train): 0.99965, score(valid) 0.12655 ::

===== finish <built-in function ctime> =====
:: score(train): mean=0.99974, std=0.00006 ::
:: score(valid): mean=0.12491, std=0.00230 ::
:: score(oof): 0.12491 ::
CPU times: user 3min 7s, sys: 12.9 s, total: 3min 20s
Wall time: 42.7 s


# Save model
- 使った特徴量名(librosa_mfcc)も一緒に保存

In [8]:
save_dir = f'{DIR_MODEL}/{NB}_{model.__class__.__name__}/'

file_dir = os.path.dirname(save_dir)
if not os.path.exists(file_dir):
    os.makedirs(file_dir)

<br>

save info

In [22]:
info_dict = {}
info_dict['featsets'] = FEATSETS
info_dict['feat_names'] = X.columns

save_path_info = f'{save_dir}info.joblib'
joblib.dump(info_dict, save_path_info)

['./../data_ignore/model/006_RandomForestClassifier/info.joblib']

<br>

save models

In [23]:
for i, model in enumerate(progress_bar(models)):
    save_path = f'{save_dir}model_{i+1}.joblib'
    joblib.dump(model, save_path, compress=9)

# Check Model

In [110]:
def make_prediction(model, signal, feat_params):
    feat, _ = extract_mfccs(signal, **feat_params)
    feat = feat[:, 1:]
    feat = feat.mean(axis=0)
    feat = feat.reshape(1, -1)
#     preds = model.predict(feat)
    preds = model.predict_proba(feat)
    preds = np.where(preds>=0.5)[1]
    
    if len(preds) == 0:
        preds_str = 'nocall'
    else:
        preds_str = list(map(lambda x: INV_BIRD_CODE[x], list(preds)))
        preds_str = ' '.join(preds_str)
    return preds_str

def load_test_clip(path, start_time, duration=5):
    signal, sr = librosa.load(path, offset=start_time, duration=duration)
    return signal, sr

In [105]:
_model = RandomForestClassifier()
save_dir = f'{DIR_MODEL}/{NB}_{_model.__class__.__name__}/'

In [59]:
path_list = sorted(glob.glob(f'{save_dir}*'))
models = []
for path in progress_bar(path_list):
    basename = os.path.basename(path)
    if 'model' in basename:
        models.append(joblib.load(path))

In [111]:
preds = []
for idx in progress_bar(range(len(test_csv))):
    # Get test row information
    site = test_csv['site'].values[idx]
    start_time = test_csv['seconds'].values[idx] - 5
    row_id = test_csv['row_id'].values[idx]
    audio_id = test_csv['audio_id'].values[idx]

    # Get the test sound clip
    if site == 'site_1' or site == 'site_2':
        signal_clip, sr = load_test_clip(DIR_TEST_AUDIO + audio_id + '.mp3', start_time)
    else:
        signal_clip, sr = load_test_clip(DIR_TEST_AUDIO + audio_id + '.mp3', 0, duration=None)

    # Make the prediction
    window_size = 0.2
    n_fft = int(sr * window_size)
    hop_length = int(n_fft/2)
    feat_params = {'sr': sr, 'n_fft': n_fft, 'hop_length': hop_length, 'n_mels': 12}
    pred = make_prediction(models[0], signal_clip, feat_params)

    # Store prediction
    preds.append([row_id, pred])
    
df_sub = pd.DataFrame(preds, columns=['row_id', 'birds'])

In [112]:
df_sub

,row_id,birds
0,site_1_41e6fe6504a34bf6846938ba78d13df1_5,nocall
1,site_1_41e6fe6504a34bf6846938ba78d13df1_10,nocall
2,site_1_41e6fe6504a34bf6846938ba78d13df1_15,nocall
3,site_1_41e6fe6504a34bf6846938ba78d13df1_20,nocall
4,site_1_41e6fe6504a34bf6846938ba78d13df1_25,nocall
...,...,...
71,site_3_9cc5d9646f344f1bbb52640a988fe902,nocall
72,site_3_a56e20a518684688a9952add8a9d5213,nocall
73,site_3_96779836288745728306903d54e264dd,nocall
74,site_3_f77783ba4c6641bc918b034a18c23e53,nocall
